<a href="https://colab.research.google.com/github/RahmiaWati431/KematanganTomatMenggunakanMachineLearning/blob/main/UjiNVBvsSVM60_40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
from IPython.display import display
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar

# --- Mount Google Drive ---
drive.mount('/content/drive')

# --- Load Data ---
svm_crop_df = pd.read_csv('/content/drive/MyDrive/Penelitian/Skripsi/Crop/60.40/Prediksi/svm/Prediksi_SVM_Crop_60:40.csv')
nb_crop_df  = pd.read_csv('/content/drive/MyDrive/Penelitian/Skripsi/Crop/60.40/Prediksi/nvb/Prediksi_NB_Crop_60:40.csv')

# --- Cek kolom ---
print("Kolom di file SVM:", svm_crop_df.columns)
print("Kolom di file NB :", nb_crop_df.columns)

# --- Ambil nama kolom prediksi secara otomatis ---
kolom_svm_pred = [col for col in svm_crop_df.columns if 'Pred' in col][0]
kolom_nb_pred  = [col for col in nb_crop_df.columns if 'Pred' in col][0]

# --- Urutkan sesuai filename ---
svm_crop_df.sort_values(by='Filename', inplace=True)
nb_crop_df.sort_values(by='Filename', inplace=True)
svm_crop_df.reset_index(drop=True, inplace=True)
nb_crop_df.reset_index(drop=True, inplace=True)

# --- Pastikan label asli sama ---
assert all(svm_crop_df['Actual'] == nb_crop_df['Actual']), "⚠ Ground truth tidak cocok!"

# --- Tabel evaluasi ---
evaluasi_df = pd.DataFrame({
    'Label Asli': svm_crop_df['Actual'],
    'SVM Crop Prediksi': svm_crop_df[kolom_svm_pred],
    'NB Crop Prediksi': nb_crop_df[kolom_nb_pred],
})

# Kolom kebenaran
evaluasi_df['SVM Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['SVM Crop Prediksi']
evaluasi_df['NB Benar?']  = evaluasi_df['Label Asli'] == evaluasi_df['NB Crop Prediksi']

# --- Hitung a, b, c, d ---
a = sum((evaluasi_df['SVM Benar?']) & (evaluasi_df['NB Benar?']))   # SVM benar, NB benar
b = sum((evaluasi_df['SVM Benar?']) & (~evaluasi_df['NB Benar?']))  # SVM benar, NB salah
c = sum((~evaluasi_df['SVM Benar?']) & (evaluasi_df['NB Benar?']))  # SVM salah, NB benar
d = sum((~evaluasi_df['SVM Benar?']) & (~evaluasi_df['NB Benar?'])) # SVM salah, NB salah

# --- 📊 Tabel Kontingensi Lengkap ---
print("\n📊 Tabel Kontingensi Lengkap (a, b, c, d):")
kontingensi_lengkap = pd.DataFrame(
    [[a, b],
     [c, d]],
    index=['SVM Benar', 'SVM Salah'],
    columns=['NB Benar', 'NB Salah']
)
display(kontingensi_lengkap)

# --- 📊 Tabel McNemar (b & c saja, urutan sama seperti kontingensi) ---
print("\n📊 Tabel McNemar (hanya b dan c, urutan konsisten):")
tabel_mcnemar = pd.DataFrame(
    [[0, b],
     [c, 0]],
    index=['SVM Benar', 'SVM Salah'],
    columns=['NB Benar', 'NB Salah']
)
display(tabel_mcnemar)

# --- Uji McNemar ---
result = mcnemar([[a, b], [c, d]], exact=True)
print("\nHasil Uji McNemar:")
print(f"Statistik: {result.statistic}")
print(f"p-value: {result.pvalue:.4f}")

# Interpretasi
alpha = 0.05
if result.pvalue < alpha:
    print("📊 Terdapat perbedaan signifikan antara model SVM Crop dan NB Crop (p < 0.05)")
else:
    print("📊 Tidak terdapat perbedaan signifikan antara model SVM Crop dan NB Crop (p >= 0.05)")

# --- Simpan tabel evaluasi ---
output_excel = '/content/drive/MyDrive/Penelitian/Skripsi/SVMvsNVB/Evaluasi_McNemar_SVM_Crop_vs_NB_Crop_60-40.xlsx'
evaluasi_df.to_excel(output_excel, index=False)
print(f"\n✅ Tabel evaluasi disimpan ke: {output_excel}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Kolom di file SVM: Index(['Filename', 'Actual', 'Predicted_SVM_Crop'], dtype='object')
Kolom di file NB : Index(['Filename', 'Actual', 'Predicted_NB_Crop'], dtype='object')

📊 Tabel Kontingensi Lengkap (a, b, c, d):


,NB Benar,NB Salah
SVM Benar,77,5
SVM Salah,0,2



📊 Tabel McNemar (hanya b dan c, urutan konsisten):


,NB Benar,NB Salah
SVM Benar,0,5
SVM Salah,0,0



Hasil Uji McNemar:
Statistik: 0.0
p-value: 0.0625
📊 Tidak terdapat perbedaan signifikan antara model SVM Crop dan NB Crop (p >= 0.05)

✅ Tabel evaluasi disimpan ke: /content/drive/MyDrive/Penelitian/Skripsi/SVMvsNVB/Evaluasi_McNemar_SVM_Crop_vs_NB_Crop_60-40.xlsx


# New Section